## import necessary lib

In [2]:
import laspy
import numpy as np
import os

def load_semantic_data(path, file):
    """Load binary point cloud data"""
    data = np.fromfile(f'{path}/{file}', dtype=np.float32)  # Use float32 (typical for LiDAR)
    if data.size == 0:
        print(f"Warning: Empty file {file}")
        return None
    
    new_size = (data.shape[0] // 4) * 4  # Ensure correct reshaping
    trimmed_data = data[:new_size]  
    reshaped_data = trimmed_data.reshape(-1, 4)  # (X, Y, Z, Intensity)
    return reshaped_data

def save_las_file(data, output_path):
    """Save processed data as a LAS file"""
    if data is None or data.size == 0:
        print(f"Skipping empty file: {output_path}")
        return
    
    x, y, z = data[:, 0], data[:, 1], data[:, 2]  # Extract XYZ
    intensity = data[:, 3]  # Extract intensity

    # Dynamically determine scale (avoid overflow)
    scale_factor = 1e-3  # 0.001 (adjustable)
    offset_x, offset_y, offset_z = np.mean(x), np.mean(y), np.mean(z)  # Use mean to center coordinates

    # Create LAS header
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.offsets = np.array([offset_x, offset_y, offset_z])
    header.scales = np.array([scale_factor, scale_factor, scale_factor])  # Smaller scale to prevent overflow

    # Create LAS file
    las = laspy.LasData(header)
    las.x, las.y, las.z = x, y, z  # Assign point cloud data
    las.intensity = intensity.astype(np.uint16)  # Convert intensity if needed

    las.write(output_path)
    print(f'Successfully saved: {output_path}')

## check available semantickitti file

In [ ]:
folders = os.listdir('./semantic_kitti/dataset/sequences/')
print(folders)
folders = folders[5:]
folders

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']


['05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21']

### save bin to las

In [16]:
for folder in folders:
    path = f'./semantic_kitti/dataset/sequences/{folder}/velodyne'
    files = os.listdir(path)
    for file in files:
        # print(path,file)
        data = load_semantic_data(path,file)
        # print(f'./output/{folder}_{file[:-4]}.las',len(data))
        save_las_file(data,f'./output/{folder}_{file[:-4]}.las')

print('all done!!!')

Successfully saved: ./output/05_000000.las
Successfully saved: ./output/05_000001.las
Successfully saved: ./output/05_000002.las
Successfully saved: ./output/05_000003.las
Successfully saved: ./output/05_000004.las
Successfully saved: ./output/05_000005.las
Successfully saved: ./output/05_000006.las
Successfully saved: ./output/05_000007.las
Successfully saved: ./output/05_000008.las
Successfully saved: ./output/05_000009.las
Successfully saved: ./output/05_000010.las
Successfully saved: ./output/05_000011.las
Successfully saved: ./output/05_000012.las
Successfully saved: ./output/05_000013.las
Successfully saved: ./output/05_000014.las
Successfully saved: ./output/05_000015.las
Successfully saved: ./output/05_000016.las
Successfully saved: ./output/05_000017.las
Successfully saved: ./output/05_000018.las
Successfully saved: ./output/05_000019.las
Successfully saved: ./output/05_000020.las
Successfully saved: ./output/05_000021.las
Successfully saved: ./output/05_000022.las
Successfull

OSError: [Errno 28] No space left on device

## check available las files

In [8]:
out_files = os.listdir('./output')
out_files

['00_000000.las',
 '00_000001.las',
 '00_000002.las',
 '00_000003.las',
 '00_000004.las',
 '00_000005.las',
 '00_000006.las',
 '00_000007.las',
 '00_000008.las',
 '00_000009.las',
 '00_000010.las',
 '00_000011.las',
 '00_000012.las',
 '00_000013.las',
 '00_000014.las',
 '00_000015.las',
 '00_000016.las',
 '00_000017.las',
 '00_000018.las',
 '00_000019.las',
 '00_000020.las',
 '00_000021.las',
 '00_000022.las',
 '00_000023.las',
 '00_000024.las',
 '00_000025.las',
 '00_000026.las',
 '00_000027.las',
 '00_000028.las',
 '00_000029.las',
 '00_000030.las',
 '00_000031.las',
 '00_000032.las',
 '00_000033.las',
 '00_000034.las',
 '00_000035.las',
 '00_000036.las',
 '00_000037.las',
 '00_000038.las',
 '00_000039.las',
 '00_000040.las',
 '00_000041.las',
 '00_000042.las',
 '00_000043.las',
 '00_000044.las',
 '00_000045.las',
 '00_000046.las',
 '00_000047.las',
 '00_000048.las',
 '00_000049.las',
 '00_000050.las',
 '00_000051.las',
 '00_000052.las',
 '00_000053.las',
 '00_000054.las',
 '00_00005

### las file format

In [ ]:
lf = laspy.open(f'./output/{out_files[0]}')
las = lf.read()
las.points

<ScaleAwarePointRecord(fmt: <PointFormat(3, 0 bytes of extra dims)>, len: 0, point size: 34)>

## bin data format

In [12]:
import numpy as np
bindata = np.fromfile('./semantic_kitti/dataset/sequences/00/velodyne/000000.bin')
bindata.reshape(-1,4)

array([[ 3.93184702e-016,  9.02218751e-012,  1.00323182e-008,
         5.30554803e-315],
       [ 1.50571510e-006,  5.30558865e-315,  4.28665954e-002,
         5.31876404e-315],
       [ 1.38376148e-001,  5.31841076e-315,  6.93055754e-001,
         5.32006731e-315],
       ...,
       [-1.10057309e-001,  7.43866508e-007, -1.02564409e-001,
         3.81469988e-007],
       [-9.75763352e-002,  5.91278308e-007, -9.40877945e-002,
         8.96454253e-007],
       [-8.46008212e-002,  1.19209375e-007, -1.32196218e-001,
         1.59106399e-314]])

## save Las to bin

In [6]:
targets = []
for file in os.listdir('2_Data_Laser/Laser/'):
    if file.endswith('.las'):
        targets.append(file)

targets.sort()
print(targets)

['2024-05-27-11-39-07_Velodyne-VLS-128-Data_out.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_1.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_10.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_11.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_12.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_13.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_14.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_15.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_16.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_17.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_18.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_19.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_2.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_20.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_21.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_22.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_23.las', '2024-05-27-11-39-07_Velodyne-VLS-128-Data_out_24.la

In [ ]:
# read las file
data_laser_path = './2_Data_Laser/Laser/'

for target in targets:
    lf = laspy.open(f'{data_laser_path}{target}')
    las = lf.read()


    point_data = list(zip(las.x, las.y, las.z,las.intensity))
    point_array = np.array(point_data, dtype=np.float64)
    point_array.tofile(f'./bin_out/{os.path.splitext(target)[0]}.bin')